In [1]:
import os
import numpy as np
import pandas as pd
import pickle

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import ShuffleSplit, GridSearchCV

pd.set_option("display.max_rows", None, "display.max_columns", None)

C:\Users\mtayl\AppData\Local\Temp\ipykernel_29144\68269814.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Gradient Boosting
In this notebook, we will be using Gradient Boosting in order to try and predict individual games. We will be using the testing and training data that was generated from the `training-data` notebook to train our model.
## Getting the data ready

In [2]:
path = os.path.abspath(f'../../data/training_set.csv')
training_df = pd.read_csv(path)
training_df['Loss'] = 1 - training_df['Win']
training_df.head()

,FG_SMA,FG_CMA,FG_EMA,FGA_SMA,FGA_CMA,FGA_EMA,FG%_SMA,FG%_CMA,FG%_EMA,3P_SMA,3P_CMA,3P_EMA,3PA_SMA,3PA_CMA,3PA_EMA,3P%_SMA,3P%_CMA,3P%_EMA,FT_SMA,FT_CMA,FT_EMA,FTA_SMA,FTA_CMA,FTA_EMA,FT%_SMA,FT%_CMA,FT%_EMA,ORB_SMA,ORB_CMA,ORB_EMA,TRB_SMA,TRB_CMA,TRB_EMA,AST_SMA,AST_CMA,AST_EMA,STL_SMA,STL_CMA,STL_EMA,BLK_SMA,BLK_CMA,BLK_EMA,TOV_SMA,TOV_CMA,TOV_EMA,PF_SMA,PF_CMA,PF_EMA,ORtg_SMA,ORtg_CMA,ORtg_EMA,DRtg_SMA,DRtg_CMA,DRtg_EMA,Pace_SMA,Pace_CMA,Pace_EMA,FTr_SMA,FTr_CMA,FTr_EMA,3PAr_SMA,3PAr_CMA,3PAr_EMA,TS%_SMA,TS%_CMA,TS%_EMA,TRB%_SMA,TRB%_CMA,TRB%_EMA,AST%_SMA,AST%_CMA,AST%_EMA,STL%_SMA,STL%_CMA,STL%_EMA,BLK%_SMA,BLK%_CMA,BLK%_EMA,eFG%_SMA,eFG%_CMA,eFG%_EMA,TOV%_SMA,TOV%_CMA,TOV%_EMA,ORB%_SMA,ORB%_CMA,ORB%_EMA,FT/FGA_SMA,FT/FGA_CMA,FT/FGA_EMA,opp_FG_SMA,opp_FG_CMA,opp_FG_EMA,opp_FGA_SMA,opp_FGA_CMA,opp_FGA_EMA,opp_FG%_SMA,opp_FG%_CMA,opp_FG%_EMA,opp_3P_SMA,opp_3P_CMA,opp_3P_EMA,opp_3PA_SMA,opp_3PA_CMA,opp_3PA_EMA,opp_3P%_SMA,opp_3P%_CMA,opp_3P%_EMA,opp_FT_SMA,opp_FT_CMA,opp_FT_EMA,opp_FTA_SMA,opp_FTA_CMA,opp_FTA_EMA,opp_FT%_SMA,opp_FT%_CMA,opp_FT%_EMA,opp_ORB_SMA,opp_ORB_CMA,opp_ORB_EMA,opp_TRB_SMA,opp_TRB_CMA,opp_TRB_EMA,opp_AST_SMA,opp_AST_CMA,opp_AST_EMA,opp_STL_SMA,opp_STL_CMA,opp_STL_EMA,opp_BLK_SMA,opp_BLK_CMA,opp_BLK_EMA,opp_TOV_SMA,opp_TOV_CMA,opp_TOV_EMA,opp_PF_SMA,opp_PF_CMA,opp_PF_EMA,opp_ORtg_SMA,opp_ORtg_CMA,opp_ORtg_EMA,opp_DRtg_SMA,opp_DRtg_CMA,opp_DRtg_EMA,opp_Pace_SMA,opp_Pace_CMA,opp_Pace_EMA,opp_FTr_SMA,opp_FTr_CMA,opp_FTr_EMA,opp_3PAr_SMA,opp_3PAr_CMA,opp_3PAr_EMA,opp_TS%_SMA,opp_TS%_CMA,opp_TS%_EMA,opp_TRB%_SMA,opp_TRB%_CMA,opp_TRB%_EMA,opp_AST%_SMA,opp_AST%_CMA,opp_AST%_EMA,opp_STL%_SMA,opp_STL%_CMA,opp_STL%_EMA,opp_BLK%_SMA,opp_BLK%_CMA,opp_BLK%_EMA,opp_eFG%_SMA,opp_eFG%_CMA,opp_eFG%_EMA,opp_TOV%_SMA,opp_TOV%_CMA,opp_TOV%_EMA,opp_ORB%_SMA,opp_ORB%_CMA,opp_ORB%_EMA,opp_FT/FGA_SMA,opp_FT/FGA_CMA,opp_FT/FGA_EMA,Neutral,Win,Loss
0,22.8,24.653846,23.211004,54.4,56.038462,55.951135,0.4192,0.440962,0.414875,5.0,6.307692,4.723021,17.4,20.076923,17.688046,0.2916,0.318923,0.265432,15.4,13.615385,12.571720,20.6,18.807692,16.287396,0.7516,0.718615,0.756023,7.8,7.423077,7.630525,30.6,29.538462,29.642729,12.0,12.307692,11.694995,5.6,6.923077,6.909948,3.2,4.230769,3.675965,12.0,11.384615,12.212031,20.2,16.769231,19.323787,96.86,100.507692,93.371986,104.72,101.753846,101.552341,68.06,68.665385,67.742130,0.3852,0.342192,0.297102,0.3228,0.355769,0.318668,0.5124,0.533500,0.498947,48.30,47.223077,47.416480,53.10,50.046154,50.683134,8.20,10.000000,10.137945,8.38,11.403846,10.344138,0.4658,0.497000,0.457629,15.76,14.819231,16.077612,25.74,23.003846,24.421789,0.2866,0.249000,0.228649,21.4,21.538462,21.057987,54.8,52.538462,54.855677,0.3910,0.409423,0.384831,8.2,8.076923,8.884101,24.2,24.846154,25.275189,0.3360,0.321192,0.350582,10.4,11.846154,11.148380,14.6,15.884615,14.613588,0.7082,0.739269,0.753357,6.8,6.115385,7.383697,31.4,30.653846,32.536725,11.8,11.653846,11.233467,3.4,4.192308,3.474109,2.8,2.346154,2.342764,11.6,11.346154,13.049714,15.0,14.653846,15.671923,92.98,96.769231,93.931371,102.02,104.400000,99.376988,65.82,65.400000,66.126001,0.2672,0.316538,0.267147,0.4408,0.468308,0.459948,0.4962,0.523346,0.502494,47.46,49.119231,49.264062,55.60,53.553846,53.411235,5.08,6.373077,5.167442,6.86,7.053846,6.029065,0.4656,0.484962,0.465742,15.52,15.811538,17.140137,22.40,20.638462,24.324966,0.1900,0.236269,0.203534,0,1,0
1,26.0,25.363636,24.750136,58.6,57.000000,56.395143,0.4496,0.446727,0.442387,4.0,5.727273,4.457340,15.0,18.681818,15.916814,0.2764,0.310500,0.282430,17.2,14.590909,17.632687,22.0,19.681818,21.997752,0.7602,0.735909,0.786252,11.4,8.272727,10.136214,34.2,32.500000,32.159791,11.0,11.681818,10.272923,5.8,5.500000,6.499974,3.0,2.909091,2.816150,10.6,10.863636,11.167679,16.4,15.454545,15.446714,108.50,103.040909,106.421923,105.94,101.304545,105.870044,67.44,68.240909,67.298025,0.3826,0.354000,0.396063,0.2590,0.328182,0.284080,0.5360,0.537273,0.539165,53.86,50.554545,51.135159,42.50,45.390909,41.533877,8.56,7.936364,9.626

In [3]:
path = os.path.abspath(f'../../data/testing_set.csv')
testing_df = pd.read_csv(path)
testing_df['Loss'] = 1 - testing_df['Win']
testing_df.head()

,FG_SMA,FG_CMA,FG_EMA,FGA_SMA,FGA_CMA,FGA_EMA,FG%_SMA,FG%_CMA,FG%_EMA,3P_SMA,3P_CMA,3P_EMA,3PA_SMA,3PA_CMA,3PA_EMA,3P%_SMA,3P%_CMA,3P%_EMA,FT_SMA,FT_CMA,FT_EMA,FTA_SMA,FTA_CMA,FTA_EMA,FT%_SMA,FT%_CMA,FT%_EMA,ORB_SMA,ORB_CMA,ORB_EMA,TRB_SMA,TRB_CMA,TRB_EMA,AST_SMA,AST_CMA,AST_EMA,STL_SMA,STL_CMA,STL_EMA,BLK_SMA,BLK_CMA,BLK_EMA,TOV_SMA,TOV_CMA,TOV_EMA,PF_SMA,PF_CMA,PF_EMA,ORtg_SMA,ORtg_CMA,ORtg_EMA,DRtg_SMA,DRtg_CMA,DRtg_EMA,Pace_SMA,Pace_CMA,Pace_EMA,FTr_SMA,FTr_CMA,FTr_EMA,3PAr_SMA,3PAr_CMA,3PAr_EMA,TS%_SMA,TS%_CMA,TS%_EMA,TRB%_SMA,TRB%_CMA,TRB%_EMA,AST%_SMA,AST%_CMA,AST%_EMA,STL%_SMA,STL%_CMA,STL%_EMA,BLK%_SMA,BLK%_CMA,BLK%_EMA,eFG%_SMA,eFG%_CMA,eFG%_EMA,TOV%_SMA,TOV%_CMA,TOV%_EMA,ORB%_SMA,ORB%_CMA,ORB%_EMA,FT/FGA_SMA,FT/FGA_CMA,FT/FGA_EMA,opp_FG_SMA,opp_FG_CMA,opp_FG_EMA,opp_FGA_SMA,opp_FGA_CMA,opp_FGA_EMA,opp_FG%_SMA,opp_FG%_CMA,opp_FG%_EMA,opp_3P_SMA,opp_3P_CMA,opp_3P_EMA,opp_3PA_SMA,opp_3PA_CMA,opp_3PA_EMA,opp_3P%_SMA,opp_3P%_CMA,opp_3P%_EMA,opp_FT_SMA,opp_FT_CMA,opp_FT_EMA,opp_FTA_SMA,opp_FTA_CMA,opp_FTA_EMA,opp_FT%_SMA,opp_FT%_CMA,opp_FT%_EMA,opp_ORB_SMA,opp_ORB_CMA,opp_ORB_EMA,opp_TRB_SMA,opp_TRB_CMA,opp_TRB_EMA,opp_AST_SMA,opp_AST_CMA,opp_AST_EMA,opp_STL_SMA,opp_STL_CMA,opp_STL_EMA,opp_BLK_SMA,opp_BLK_CMA,opp_BLK_EMA,opp_TOV_SMA,opp_TOV_CMA,opp_TOV_EMA,opp_PF_SMA,opp_PF_CMA,opp_PF_EMA,opp_ORtg_SMA,opp_ORtg_CMA,opp_ORtg_EMA,opp_DRtg_SMA,opp_DRtg_CMA,opp_DRtg_EMA,opp_Pace_SMA,opp_Pace_CMA,opp_Pace_EMA,opp_FTr_SMA,opp_FTr_CMA,opp_FTr_EMA,opp_3PAr_SMA,opp_3PAr_CMA,opp_3PAr_EMA,opp_TS%_SMA,opp_TS%_CMA,opp_TS%_EMA,opp_TRB%_SMA,opp_TRB%_CMA,opp_TRB%_EMA,opp_AST%_SMA,opp_AST%_CMA,opp_AST%_EMA,opp_STL%_SMA,opp_STL%_CMA,opp_STL%_EMA,opp_BLK%_SMA,opp_BLK%_CMA,opp_BLK%_EMA,opp_eFG%_SMA,opp_eFG%_CMA,opp_eFG%_EMA,opp_TOV%_SMA,opp_TOV%_CMA,opp_TOV%_EMA,opp_ORB%_SMA,opp_ORB%_CMA,opp_ORB%_EMA,opp_FT/FGA_SMA,opp_FT/FGA_CMA,opp_FT/FGA_EMA,Neutral,Win,Loss
0,24.6,25.000000,26.295223,57.2,58.588235,59.950459,0.4262,0.427588,0.434373,7.4,6.588235,8.204161,23.6,22.176471,24.070312,0.3084,0.299059,0.337075,14.0,12.647059,13.172682,20.8,19.235294,19.369426,0.6766,0.650882,0.685984,8.0,8.705882,8.663763,33.0,32.000000,33.494042,15.2,15.058824,16.393277,6.0,6.529412,7.253464,4.0,3.529412,3.867263,12.8,13.588235,12.652723,18.6,19.529412,18.925827,99.60,96.376471,102.177531,103.68,99.011765,100.365633,71.06,72.029412,72.268344,0.3742,0.336706,0.333050,0.4134,0.378235,0.402471,0.5246,0.512118,0.532588,51.98,51.194118,53.618324,61.28,59.847059,61.491816,8.30,8.976471,9.928151,12.04,10.111765,12.098830,0.4896,0.483588,0.501728,16.10,16.652941,15.522065,25.48,26.558824,27.283188,0.2510,0.221882,0.224880,25.6,26.263158,24.161796,54.6,59.578947,54.535112,0.4650,0.441000,0.440586,7.6,7.368421,6.938694,19.8,20.789474,19.170009,0.3640,0.353421,0.346890,13.8,12.157895,14.134105,21.6,19.684211,23.175522,0.6444,0.626000,0.617173,9.4,10.947368,10.232045,34.4,36.736842,37.302457,16.4,16.052632,15.467661,11.8,9.263158,10.163377,4.6,5.315789,5.158201,19.2,16.263158,19.961327,19.2,19.315789,19.325415,97.32,97.121053,92.661265,81.94,84.326316,76.493108,74.54,74.047368,75.133121,0.4000,0.335579,0.429435,0.3626,0.350158,0.351612,0.5588,0.523474,0.528957,56.72,55.089474,58.697837,65.74,61.347368,65.274972,15.62,12.384211,13.507466,12.94,15.321053,14.292547,0.5340,0.503316,0.503701,22.70,19.210526,23.150352,32.98,32.400000,33.972093,0.2552,0.206421,0.261465,0,0,1
1,25.4,25.250000,26.894015,54.8,55.916667,55.720876,0.4620,0.454417,0.480009,7.4,6.250000,7.206811,22.2,19.000000,20.849769,0.3374,0.326417,0.345893,14.2,14.500000,15.147651,17.4,19.000000,18.813979,0.8158,0.761500,0.804623,5.4,7.166667,5.983827,29.0,31.083333,31.540353,13.6,11.250000,14.659644,8.0,7.166667,7.526371,5.8,6.166667,5.913450,12.8,13.000000,12.254156,16.0,16.416667,14.752449,104.66,101.841667,108.833516,95.44,92.383333,91.727737,69.40,69.533333,69.843951,0.3280,0.349167,0.350921,0.4048,0.344750,0.374650,0.5724,0.552167,0.587170,48.32,49.283333,50.794579,52.06,43.458333,51.963275,11.52,10.166667,10

In [4]:
train_win_true, test_win_true = training_df.pop('Win'), testing_df.pop('Win')
train_loss_true, test_loss_true = training_df.pop('Loss'), testing_df.pop('Loss')

print(f'{len(training_df)} train examples')
print(f'{len(testing_df)} test examples')

12847 train examples
5507 test examples


## Training the model

In [5]:
# Define the Random Forest model
gbc = GradientBoostingClassifier()

# Define the hyperparameter grid
param_grid = {
    'loss': ['log_loss', 'deviance', 'exponential'],  # Loss function
    'learning_rate': [0.01, 0.05, 0.1],  # Learning rate (eta)
    'max_depth': [3, 5, 8],  # Maximum depth of each tree
    'max_features': ['log2', 'sqrt'],  # Maximum number of features considered for splitting
    'n_estimators': [100, 200, 300]  # Number of trees
}

# Create a single split
cv_single_split = ShuffleSplit(n_splits=1)

### Using Win as label

In [6]:
# Create the grid search object
grid_search_win = GridSearchCV(gbc, param_grid, cv=cv_single_split, verbose=5)

# Fit the grid search to the data
grid_search_win.fit(training_df, train_win_true)

# Get the best hyperparameters
best_params = grid_search_win.best_params_
print("Best hyperparameters:", best_params)

Fitting 1 folds for each of 162 candidates, totalling 162 fits
[CV 1/1] END learning_rate=0.01, loss=log_loss, max_depth=3, max_features=log2, n_estimators=100;, score=0.637 total time=   2.0s
[CV 1/1] END learning_rate=0.01, loss=log_loss, max_depth=3, max_features=log2, n_estimators=200;, score=0.665 total time=   3.9s
[CV 1/1] END learning_rate=0.01, loss=log_loss, max_depth=3, max_features=log2, n_estimators=300;, score=0.683 total time=   5.9s
[CV 1/1] END learning_rate=0.01, loss=log_loss, max_depth=3, max_features=sqrt, n_estimators=100;, score=0.645 total time=   3.6s
[CV 1/1] END learning_rate=0.01, loss=log_loss, max_depth=3, max_features=sqrt, n_estimators=200;, score=0.669 total time=   7.0s
[CV 1/1] END learning_rate=0.01, loss=log_loss, max_depth=3, max_features=sqrt, n_estimators=300;, score=0.695 total time=  10.6s
[CV 1/1] END learning_rate=0.01, loss=log_loss, max_depth=5, max_features=log2, n_estimators=100;, score=0.650 total time=   3.3s
[CV 1/1] END learning_rate=

c:\Users\mtayl\anaconda3\envs\mlmb-utils\Lib\site-packages\sklearn\model_selection\_validation.py:542: FitFailedWarning: 
54 fits failed out of a total of 162.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
54 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\mtayl\anaconda3\envs\mlmb-utils\Lib\site-packages\sklearn\model_selection\_validation.py", line 890, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\mtayl\anaconda3\envs\mlmb-utils\Lib\site-packages\sklearn\base.py", line 1344, in wrapper
    estimator._validate_params()
  File "c:\Users\mtayl\anaconda3\envs\mlmb-utils\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter

Best hyperparameters: {'learning_rate': 0.05, 'loss': 'exponential', 'max_depth': 3, 'max_features': 'sqrt', 'n_estimators': 300}


In [7]:
# Acess the best model
clf = grid_search_win.best_estimator_

y_pred = clf.predict(testing_df)

accuracy = accuracy_score(test_win_true, y_pred)
print(f"Accuracy: {(accuracy*100):.2f}")

print("\nClassification Report:")
print(classification_report(test_win_true, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(test_win_true, y_pred))

Accuracy: 68.11

Classification Report:
              precision    recall  f1-score   support

           0       0.63      0.47      0.54      2180
           1       0.70      0.82      0.76      3327

    accuracy                           0.68      5507
   macro avg       0.67      0.65      0.65      5507
weighted avg       0.67      0.68      0.67      5507


Confusion Matrix:
[[1029 1151]
 [ 605 2722]]


### Using Loss as label

In [9]:
# Create the grid search object
grid_search_loss = GridSearchCV(gbc, param_grid, cv=cv_single_split, verbose=5)

# Fit the grid search to the data
grid_search_loss.fit(training_df, train_loss_true)

# Get the best hyperparameters
best_params = grid_search_loss.best_params_
print("Best hyperparameters:", best_params)

Fitting 1 folds for each of 162 candidates, totalling 162 fits
[CV 1/1] END learning_rate=0.01, loss=log_loss, max_depth=3, max_features=log2, n_estimators=100;, score=0.613 total time=   2.0s
[CV 1/1] END learning_rate=0.01, loss=log_loss, max_depth=3, max_features=log2, n_estimators=200;, score=0.654 total time=   4.0s
[CV 1/1] END learning_rate=0.01, loss=log_loss, max_depth=3, max_features=log2, n_estimators=300;, score=0.665 total time=   5.9s
[CV 1/1] END learning_rate=0.01, loss=log_loss, max_depth=3, max_features=sqrt, n_estimators=100;, score=0.627 total time=   3.5s
[CV 1/1] END learning_rate=0.01, loss=log_loss, max_depth=3, max_features=sqrt, n_estimators=200;, score=0.654 total time=   7.0s
[CV 1/1] END learning_rate=0.01, loss=log_loss, max_depth=3, max_features=sqrt, n_estimators=300;, score=0.668 total time=  10.5s
[CV 1/1] END learning_rate=0.01, loss=log_loss, max_depth=5, max_features=log2, n_estimators=100;, score=0.640 total time=   3.3s
[CV 1/1] END learning_rate=

c:\Users\mtayl\anaconda3\envs\mlmb-utils\Lib\site-packages\sklearn\model_selection\_validation.py:542: FitFailedWarning: 
54 fits failed out of a total of 162.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
54 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\mtayl\anaconda3\envs\mlmb-utils\Lib\site-packages\sklearn\model_selection\_validation.py", line 890, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\mtayl\anaconda3\envs\mlmb-utils\Lib\site-packages\sklearn\base.py", line 1344, in wrapper
    estimator._validate_params()
  File "c:\Users\mtayl\anaconda3\envs\mlmb-utils\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter

Best hyperparameters: {'learning_rate': 0.05, 'loss': 'exponential', 'max_depth': 3, 'max_features': 'log2', 'n_estimators': 300}


In [10]:
# Acess the best model
clf = grid_search_loss.best_estimator_

y_pred = clf.predict(testing_df)

accuracy = accuracy_score(test_loss_true, y_pred)
print(f"Accuracy: {(accuracy*100):.2f}")

print("\nClassification Report:")
print(classification_report(test_loss_true, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(test_loss_true, y_pred))

Accuracy: 68.60

Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.82      0.76      3327
           1       0.64      0.48      0.55      2180

    accuracy                           0.69      5507
   macro avg       0.67      0.65      0.65      5507
weighted avg       0.68      0.69      0.68      5507


Confusion Matrix:
[[2742  585]
 [1144 1036]]


# Saving the models

In [12]:
for pair in [('win', grid_search_win.best_estimator_), ('loss', grid_search_loss.best_estimator_)]:
    filename = f'gradient_boosting_{pair[0]}_model.pkl'
    path = os.path.abspath(f'../../machine-learning/model/{filename}')
    pickle.dump(pair[1], open(path, 'wb'))

: 